Multivariate Gaussian Distribution

https://en.wikipedia.org/wiki/Multivariate_normal_distribution 

https://www.youtube.com/watch?v=JjB58InuTqM&index=6&list=LLyG2f9f44zZhZr8cLPOa4UQ&t=1s

Note:

https://photos.app.goo.gl/Dp8RB1ugEcy5FEEM8

In [1]:
import os
import pandas as pd
import numpy as np
from glob import glob

# Data preprocessing

In [2]:
path_list = glob(os.path.join(os.getcwd(), 'dataset', '*'))
path_list

['/Users/chia-yuanchang/My_Project/Machine_Learning_NTU/2_predict_income_by_logistic_regression_and_generative_model/dataset/Y_train',
 '/Users/chia-yuanchang/My_Project/Machine_Learning_NTU/2_predict_income_by_logistic_regression_and_generative_model/dataset/test.csv',
 '/Users/chia-yuanchang/My_Project/Machine_Learning_NTU/2_predict_income_by_logistic_regression_and_generative_model/dataset/X_train',
 '/Users/chia-yuanchang/My_Project/Machine_Learning_NTU/2_predict_income_by_logistic_regression_and_generative_model/dataset/train.csv',
 '/Users/chia-yuanchang/My_Project/Machine_Learning_NTU/2_predict_income_by_logistic_regression_and_generative_model/dataset/X_test',
 '/Users/chia-yuanchang/My_Project/Machine_Learning_NTU/2_predict_income_by_logistic_regression_and_generative_model/dataset/sample_submission.csv']

In [3]:
path_Y_train = path_list[0]
df_Y_train = pd.read_csv(path_Y_train)
# df_Y_train

In [4]:
path_X_train = path_list[2]
df_X_train = pd.read_csv(path_X_train)
# df_X_train

In [5]:
# list(df_X_train.columns)

In [6]:
path_X_test = path_list[4]
df_X_test = pd.read_csv(path_X_test)
# df_X_test

In [7]:
path_train = path_list[3]
df_train = pd.read_csv(path_train)
# df_train

In [8]:
path_test = path_list[1]
df_test = pd.read_csv(path_test)
# df_test

In [9]:
def normalize(df_array):
    mean = df_array.mean(axis=0)
    std = df_array.std(axis=0)
    
    # To prevent from divide by inf: if std = 0, replace it with 1
    std[std == 0] = 1
    
    df_array_normalized = (df_array - mean) / std
    return df_array_normalized

In [10]:
y_train = df_Y_train.values
x_train = df_X_train.values
x_test = df_X_test.values
y_train.shape, x_train.shape, x_test.shape

((32561, 1), (32561, 106), (16281, 106))

In [11]:
y_train = y_train.astype('float')
x_train = x_train.astype('float')
x_test = x_test.astype('float')

y_train = y_train.reshape(-1)

x_train = normalize(x_train)
x_test = normalize(x_test)

print(y_train.shape, x_train.shape, x_test.shape)
print(y_train[0], x_train[0,0], x_test[0,0])

(32561,) (32561, 106) (16281, 106)
0.0 0.030670557354391753 -0.9941292592084697


In [12]:
def shuffle(arr1, arr2):
    id_list = np.arange(len(arr1))
    np.random.shuffle(id_list)
    
    arr1_random = arr1[id_list]
    arr2_random = arr2[id_list]
    
    return arr1_random, arr2_random

x_train, y_train = shuffle(x_train, y_train)
x_train.shape, y_train.shape

((32561, 106), (32561,))

# train split to (train, val)

In [13]:
# split (train, val)
split_percent = 0.1
num_sample = len(y_train)
num_sample_train = int(num_sample * (1 - split_percent))
x_train_split = x_train[:num_sample_train]
y_train_split = y_train[:num_sample_train]
x_val_split = x_train[num_sample_train:]
y_val_split = y_train[num_sample_train:]
x_train_split.shape, y_train_split.shape, x_val_split.shape, y_val_split.shape

((29304, 106), (29304,), (3257, 106), (3257,))

# training data split two groups

y = 1, group 1

y = 0, group 2 !!!

In [14]:
x_group_1 = []
y_group_1 = []
x_group_2 = []
y_group_2 = []

for i in range(len(y_train_split)):
    if y_train_split[i] == 1:
        x_group_1.append(x_train_split[i])
        y_group_1.append(y_train_split[i])
    else:
        x_group_2.append(x_train_split[i])
        y_group_2.append(y_train_split[i])
        
x_group_1 = np.array(x_group_1)
y_group_1 = np.array(y_group_1)
x_group_2 = np.array(x_group_2)
y_group_2 = np.array(y_group_2)

x_group_1.shape, y_group_1.shape, x_group_2.shape, y_group_2.shape

((7023, 106), (7023,), (22281, 106), (22281,))

# calculate mu and sigma

In [17]:
mu1 = np.mean(x_group_1, axis=0)
mu2 = np.mean(x_group_2, axis=0)

print(mu1[0], mu2[0])
print(mu1.shape, mu2.shape)

0.4181739643467947 -0.13025452921487762
(106,) (106,)


In [22]:
x_group_1_norm = x_group_1 - mu1
x_group_2_norm = x_group_2 - mu2

print(x_group_1_norm.shape, x_group_2_norm.shape)
print(x_group_1[0,0], x_group_2[0,0])
print(x_group_1_norm[0,0], x_group_2_norm[0,0])

(7023, 106) (22281, 106)
-0.11595461046477588 0.323920892992727
-0.5341285748115706 0.45417542220760465


In [23]:
n1 = len(x_group_1_norm)
n2 = len(x_group_2_norm)

n1,n2

(7023, 22281)

In [33]:
x_train_shape = x_train.shape
dim = x_train.shape[1]

print(x_train_shape)
print(dim)

(32561, 106)
106


In [46]:
# numpy do the matrix transpose automatically
print(x_group_1_norm[:,0] @ x_group_1_norm[:,1])
print(x_group_1_norm[:,0] @ x_group_1_norm[:,1].transpose())

-355.5226572180194
-355.5226572180194


In [34]:
sigma1 = np.zeros(shape=[dim,dim])
sigma2 = np.zeros(shape=[dim,dim])

for i in range(dim):
    for j in range(i, dim):
        
        # group 1
        xi = x_group_1_norm[:,i]
        xj = x_group_1_norm[:,j]
        sigma1[i,j] = (xi @ xj) / n1
        sigma1[j,i] = sigma1[i,j]
        
        # group 2
        xi = x_group_2_norm[:,i]
        xj = x_group_2_norm[:,j]
        sigma2[i,j] = (xi @ xj) / n2
        sigma2[j,i] = sigma2[i,j]
        
print(sigma1.shape, sigma2.shape)

(106, 106) (106, 106)


In [47]:
# merge two group's sigma by weight
sigma = (n1 / (n1+n2)) * sigma1 + (n2 / (n1+n2)) * sigma2

In [49]:
sigma_inverse = np.linalg.inv(sigma)
sigma_inverse.shape

(106, 106)

# Calculate w and b

In [50]:
w = (mu1 - mu2) @ sigma_inverse
w.shape

(106,)

In [62]:
b = (
    - 0.5 * mu1.transpose() @ sigma_inverse @ mu1
    - 0.5 * mu2.transpose() @ sigma_inverse @ mu2
    + np.log(n1/n2)
)

b

-2.165555638284293

# Validate

In [63]:
z_val_predict = x_val_split @ w + b
# (3257,) = (3257, 106) @ (106,) + (1,)

z_val_predict.shape, z_val_predict[:2], (x_val_split @ w)[:2], b

((3257,),
 array([ 0.26439698, -1.42809348]),
 array([2.42995262, 0.73746216]),
 -2.165555638284293)

In [64]:
def sigmoid(z):
    res = 1 / (1.0 + np.exp(-z))
    return np.clip(res, 1e-8, 1-(1e-8))

y_val_predict = sigmoid(z_val_predict)
y_val_predict.shape, y_val_predict[:5]

((3257,), array([0.56571686, 0.19339592, 0.33341071, 0.00299755, 0.0175906 ]))

# Calculate acc

In [68]:
y_val_predict_category = np.round(y_val_predict)
compare = (y_val_predict_category == y_val_split)
accuracy = np.sum(compare) / len(y_val_split)
accuracy

0.8342032545287074

# Calculate cross entropy

In [71]:
y_real_0 = y_val_split
y_real_1 = 1 - y_val_split
y_predict_0 = y_val_predict
y_predict_1 = 1 - y_val_predict
cross_entropy = - y_real_0 * np.log(y_predict_0) - y_real_1 * np.log(y_predict_1)
mean_cross_entropy = np.mean(cross_entropy)
mean_cross_entropy

0.3450664073147981